In [7]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [8]:
# from rs import skill_check

# def test_skill_check(metadata):
#     print(skill_check("sanity", None, metadata))

# metadata = {
#     "character": {"sanity":30},
#     "symptom":[
#         "incoherent speech",
#         "uncontrollable twitching, trembling",
#         "delusions of persecution",
#         "strange appetites (dirt, clay, etc...)",
#         "scratches, punches, and bruises all over the body"
#     ]
# }

# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)

# print(metadata["character"])


In [9]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    
    # print(config.metadata["character"])
    
    if "tags" in config.metadata["character"] and "insane" in config.metadata["character"]["tags"]:
        return f"TURN: {count + 1} {is_final} (I'm insane!)"
    
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    # print(config.metadata["character"])
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        config.metadata["memories"] = []
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/coc_lot_3636.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 4
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))



Professor Dagger examines the seance table with utmost concentration and scrutiny, being as careful as possible to not inflict any damage to the table. He then shifted his attention to the three leather bound books, inspecting its condition while flipping through its contents to verify its authenticity. 

After much inspection, he turned his attention towards you, and inquired about a fourth book, expressing that altogether, it could fetch a price well above $2,000,000. Your eyes lit up, recognizing this to be your chance to turn your life around, pay off your debts and perhaps start something by yourself. The mystery of the fourth book piqued your interest despite the disinterest in the occult, and such curiosity beckoned you to ask about the importance of the fourth book. A heavy silence ensued, the Professor’s shadowy face under the chandelier’s soft  glow gazed into your eyes with great earnestness. In a somber tone, he explained that the fourth book is the rarest of them all, fo

I select option 3. TURN: 1 


You glare at Professor Dagger, the grip on your steering wheel tightening as you let out a stern warning, "Listen, if you've lied about the money, I swear I'll make you regret it." The tension in the car thickens, a stark contrast to the dimly lit streets you're navigating through.

Professor Dagger, maintaining his composure, raises his hands in a calming gesture, "I assure you, my intentions are genuine. But what I'm about to reveal may add layers to our current quest." He pauses, ensuring he has your full attention. "The previous owner of your storage, along with his family, was deeply involved with the occult. It's said that they conducted an abominable ritual with the fourth book, summoning a demon to possess her sister. The aftermath of those events are shrouded in mystery, whispers, and speculation."

As Dagger finishes his confession, a chill runs down your spine, not from fear but from the realization of what might lie within the storage you've ac


Memories:
"""
- Kyle, influenced by the potential of a significant financial gain, warns Professor Dagger about the consequences of lying about the money. Professor Dagger confesses that the previous owner of the storage was deeply involved in the occult, hinting at the dangerous and mysterious contents that might be inside. The round ends as they arrive at the self-storage facility, ready to uncover the truths hidden within.
"""


You invited Professor Dagger into the storage room, the space still cluttered despite your earlier attempt to clear some of the items out. The two of you began scouring through old furniture, leaving nothing unturned, yet, after two long hours, your hands held nothing but dust. You sensed the Professor’s eyes laced with disappointment, perhaps he has already given up the search, for to encounter a rare artifact like the one he described must require great deal of luck. But your desire for a better life fuels your persistence to hunt the fourth book down. Yo

I select option 2. TURN: 1 


You can't help but wrinkle your nose, the stench growing stronger with each step you take down the corridor. "What is that foul odor?" you ask, trying to keep your voice steady despite the growing unease.

Professor Dagger pauses for a moment, his lantern casting long shadows on the walls. "Decay," he says simply, his voice a mix of fear and fascination. "The smell of ancient secrets and... consequences. We must be cautious."

The corridor seems to stretch endlessly before you, the air thick with anticipation and dread. You notice strange symbols etched into the walls, their meanings lost to time but their intent unmistakably ominous.

As you proceed, the light from Professor Dagger's lantern reveals the end of the corridor. A door stands before you, its surface covered in more cryptic symbols. Professor Dagger reaches out to open it, his hand trembling slightly.

What do you do as he opens the door?

possible actions:
1. Brace yourself and look away
2. Pe

I select option 2. TURN: 2 


Curiosity gets the better of you, and as Professor Dagger's hand pushes the door open, you peer over his shoulder to catch a glimpse of what lies beyond. The door creaks ominously, revealing a room that is unlike any other part of the storage facility. It's dimly lit by candles placed in a circle around a central figure on the floor. 

Your eyes are immediately drawn to the near-withered body lying at the center of the room, over a pentagram made of a strange red fluid. The sight is horrifying, and you can't help but notice that the withered body's face is completely gone, leaving a gaping cavity where something sinister seems to writhe inside.

You need to make a skill check to maintain your sanity upon witnessing this grotesque scene.

skill: sanity
difficulty: hard

I am making a SANITY check against my own sanity points: 10.
And... FAILED! I lost 6 sanity points.
I am shocked and lose control of my actions.
TURN: 3  (I'm insane!)


As the grotesque reality of the scene before you fully registers, your mind reels from the shock. The loss of sanity points reflects your psyche fracturing under the weight of what you've witnessed. The world around you blurs, sounds distort, and for a moment, reality itself seems to warp and twist.

In your frenzied state, you stagger backward, your breath coming in short, ragged gasps. The walls of the room seem to close in on you, the flickering candlelight casting menacing shadows that dance and twist in your peripheral vision. The writhing entity within the withered body's facial cavity appears to grow, its movements more frantic as if sensing your distress.

Professor Dagger, noticing your distress, attempts to calm you, his words barely piercing the veil of your hysteria. "Kyle! Focus on my voice. You must not l

grab the book and run. TURN: 4  (I'm insane!)


Driven by a mix of fear, desperation, and an inexplicable urge, you lunge towards the podium at the far end of the hidden room, your hands grasping the fourth book. The moment your fingers touch the spine of the book, the strange red fluid forming the pentagram on the floor begins to evaporate, disappearing without a trace.

As the fluid vanishes, the room's atmosphere shifts dramatically. The withered body starts to convulse violently, tentacles pouring out from the gaping cavity that was once its face. They lash out in all directions, and before you can process what's happening, they ensnare Professor Dagger, pulling him towards the writhing mass. His screams fill the room, a chilling reminder of the horror unfolding before you.

With the book in hand, you turn and run, fueled by adrenaline and terror. The corridor that once led you to this nightmare now seems like a lifeline back to reality. You can hear the slithering and wet sounds o

I am making a skill check using DEXTERITY against a difficulty of HARD.
And I rolled a 32 for a result of FAILURE.
TURN: 5  (I'm insane!)


In your panicked state, your feet seem to betray you. Your movements are erratic, hindered by the overwhelming fear and the disorientation from your recent loss of sanity. As you attempt to sprint down the corridor, your foot catches on an uneven part of the floor, sending you sprawling forward. The heavy thud of your body against the cold ground echoes in the narrow space, a stark reminder of your vulnerability.

Before you can fully recover, you feel the chilling touch of the tentacles as they brush against your ankle. The sensation is cold, wet, and utterly revolting, sending shivers up your spine. With a surge of desperate strength, you kick at the tentacle and scramble to your feet, continuing your frenzied dash towards the exit.

The creature, relentless in its pursuit, seems to be slowed down momentarily by your actions, but the sounds of it

no. TURN: 6  (I'm insane!)


With no prior arrangement made with the middle-aged woman to unlock the door, you find yourself trapped, the locked exit standing between you and your escape from the nightmarish entity that is relentlessly pursuing you. The realization hits you like a physical blow, your heart pounding in your chest as you frantically search for any means to unlock the door or find another way out.

The sounds of the creature's approach grow louder, a cacophony of slithering and the wet slap of its tentacles against the ground. You can almost feel its breath on your neck, the air growing colder as it draws near.

In a last-ditch effort, you examine the door for any weaknesses, any overlooked detail that might offer a way out. Your hands shake as you try the handle again, hoping against hope for a miracle.

But it's clear, there's no easy way out. You need to make a skill check to find an alternative solution or create an opportunity to escape.

skill: spot_hidden
difficult

I am making a skill check using SPOT_HIDDEN against a difficulty of HARD.
And I rolled a 4 for a result of CRITICAL SUCCESS.
TURN: 7  (I'm insane!)


Your critical success in the spot hidden check, even in the throes of insanity, sharpens your senses to a razor's edge. Amidst the chaos of your mind and the imminent threat lurking just moments behind you, a glimmer of hope catches your eye. 

Hidden beneath a loose floorboard near the door, almost imperceptible unless one were looking for it with desperate urgency, you find a set of old keys. With trembling hands, you grab them, trying each one in the lock with frantic speed. The creature's grotesque sounds of pursuit are nearly upon you, its malevolent presence a palpable force driving you to the brink of despair.

Finally, one of the keys turns with a satisfying click, and the door swings open just as you feel the creature's tendrils graze the back of your neck. You hurl yourself through the doorway, slamming it shut behind you and lo